# データベース確認サンプル

このノートブックではデータベースの中身を確認する方法を示します。

In [ ]:
# 必要なライブラリのインポート
import sys
import os
from pathlib import Path

# fastapiディレクトリをPythonパスに追加
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
from sqlalchemy import text
from database import SessionLocal, engine
from models.user import User
from models.mountain import Mountain, Type, Prefecture
from models.path import Path, PathGeometry, PathTag

In [ ]:
# SQLクエリで直接データを取得
query = """
SELECT 
    m.name,
    m.elevation,
    p.name as prefecture
FROM mountains m
LEFT JOIN mountain_prefectures mp ON m.id = mp.mountain_id
LEFT JOIN prefectures p ON mp.prefecture_id = p.id
ORDER BY m.elevation DESC NULLS LAST
LIMIT 10;
"""

df = pd.read_sql(query, engine)
print("標高が高い山トップ10:")
print(df)

In [ ]:
# カスタムクエリを実行するヘルパー関数
def query_to_df(query_string):
    """SQLクエリを実行してDataFrameを返す"""
    return pd.read_sql(query_string, engine)

def show_table_info(table_name):
    """テーブルの構造とサンプルデータを表示"""
    # カラム情報を取得
    query = f"""
    SELECT 
        column_name,
        data_type,
        is_nullable
    FROM information_schema.columns
    WHERE table_name = '{table_name}'
    ORDER BY ordinal_position;
    """
    print(f"テーブル: {table_name}")
    print("\nカラム情報:")
    print(query_to_df(query))
    
    # サンプルデータを表示
    print(f"\nサンプルデータ（最初の5件）:")
    sample_query = f"SELECT * FROM {table_name} LIMIT 5;"
    print(query_to_df(sample_query))

# 使用例
# show_table_info('mountains')

## 8. カスタムクエリ用のヘルパー関数

## 7. SQLクエリを直接実行する例

In [ ]:
# 特定のPathの詳細情報を取得
db = SessionLocal()
try:
    path = db.query(Path).first()
    
    if path:
        print(f"Path ID: {path.id}")
        print(f"OSM ID: {path.osm_id}")
        print(f"Type: {path.type}")
        print(f"\nタグ情報:")
        for tag in path.tags:
            print(f"  - highway: {tag.highway}, source: {tag.source}")
        
        print(f"\n座標データ（最初の10件）:")
        df_geometries = pd.DataFrame([
            {
                'node_id': g.node_id,
                'lat': g.lat,
                'lon': g.lon,
                'sequence': g.sequence,
            }
            for g in path.geometries[:10]
        ])
        print(df_geometries)
    else:
        print("Pathデータが存在しません")
finally:
    db.close()

## 6. Pathの座標データとタグ情報を確認

In [ ]:
# Pathsテーブルの最初の5件を取得
db = SessionLocal()
try:
    paths = db.query(Path).limit(5).all()
    
    df_paths = pd.DataFrame([
        {
            'id': p.id,
            'osm_id': p.osm_id,
            'type': p.type,
            'minlat': p.minlat,
            'minlon': p.minlon,
            'maxlat': p.maxlat,
            'maxlon': p.maxlon,
        }
        for p in paths
    ])
    
    print(df_paths)
finally:
    db.close()

## 5. Pathsテーブルのデータを確認

In [ ]:
# リレーションを含む山の情報を取得
db = SessionLocal()
try:
    mountains = db.query(Mountain).limit(5).all()
    
    for mountain in mountains:
        print(f"\n{'='*60}")
        print(f"山名: {mountain.name} ({mountain.yomi})")
        print(f"標高: {mountain.elevation}m")
        print(f"座標: ({mountain.lat}, {mountain.lon})")
        
        # 関連する都道府県を表示
        if mountain.prefectures:
            prefs = ', '.join([p.name for p in mountain.prefectures])
            print(f"都道府県: {prefs}")
        
        # 関連するタイプを表示
        if mountain.types:
            types = ', '.join([t.name for t in mountain.types])
            print(f"タイプ: {types}")
finally:
    db.close()

## 4. 山の情報とリレーションを確認（Type, Prefecture含む）

In [ ]:
# Mountainsテーブルの最初の10件を取得
db = SessionLocal()
try:
    mountains = db.query(Mountain).limit(10).all()
    
    # pandasのDataFrameに変換
    df_mountains = pd.DataFrame([
        {
            'id': m.id,
            'name': m.name,
            'yomi': m.yomi,
            'elevation': m.elevation,
            'lat': m.lat,
            'lon': m.lon,
            'area': m.area,
        }
        for m in mountains
    ])
    
    print(df_mountains)
finally:
    db.close()

## 3. Mountainsテーブルのデータを確認

In [ ]:
db = SessionLocal()
try:
    counts = {
        'users': db.query(User).count(),
        'mountains': db.query(Mountain).count(),
        'types': db.query(Type).count(),
        'prefectures': db.query(Prefecture).count(),
        'paths': db.query(Path).count(),
        'path_geometries': db.query(PathGeometry).count(),
        'path_tags': db.query(PathTag).count(),
    }
    
    df_counts = pd.DataFrame(list(counts.items()), columns=['テーブル名', 'レコード数'])
    print(df_counts)
finally:
    db.close()

## 2. 各テーブルのレコード数を確認

In [ ]:
# データベース接続を確認
db = SessionLocal()
try:
    result = db.execute(text("SELECT version();"))
    version = result.scalar()
    print(f"PostgreSQL version: {version}")
finally:
    db.close()

## 1. データベース接続の確認